# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  Jul  5 2022  3:04PM  241133        10       MSP211876   
1  Jul  5 2022  3:04PM  241133        10       MSP211877   
2  Jul  5 2022  3:04PM  241133        10       MSP211879   
3  Jul  5 2022  3:04PM  241133        10       MSP211874   
4  Jul  5 2022  3:04PM  241133        10       MSP211875   
5  Jul  5 2022  3:04PM  241133        10       MSP211885   
6  Jul  5 2022  3:04PM  241133        10       MSP211886   
7  Jul  5 2022  2:58PM  241125        10      CTF0011298   
8  Jul  5 2022  2:58PM  241125        10      CTF0011299   
9  Jul  5 2022  2:57PM  241123        19  ACG-2102490522   

                Customer ShipmentStatus  
0       Methapharm, Inc.       Released  
1       Methapharm, Inc.       Released  
2       Methapharm, Inc.       Released  
3       Methapharm, Inc.       Released  
4       Methapharm, Inc.       Released  
5       Methapharm, Inc.       Released  
6       Methapharm, Inc.       Released  
7         Citieffe, Inc.       Released  
8         Citieffe, Inc.       Released  
9  ACG North America LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
67  241120       Released         12
68  241122       Released          1
69  241123       Released          1
70  241125       Released          2
71  241133       Released          7

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
241120                NaN        NaN      12.0
241122                NaN        NaN       1.0
241123                NaN        NaN       1.0
241125                NaN        NaN       2.0
241133                NaN        NaN       7.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241010                0.0       35.0      25.0
241016                0.0        0.0      60.0
241017                0.0        0.0      60.0
241019                0.0        0.0      55.0
241022               59.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241010                  0         35        25
241016                  0          0        60
241017                  0          0        60
241019                  0          0        55
241022                 59          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               241010          0         35        25
1               241016          0          0        60
2               241017          0          0        60
3               241019          0          0        55
4               241022         59          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               241010                  35       25
1               241016                           60
2               241017                           60
3               241019                           55
4               241022        59         1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0   Jul  5 2022  3:04PM  241133        10       Methapharm, Inc.
7   Jul  5 2022  2:58PM  241125        10         Citieffe, Inc.
9   Jul  5 2022  2:57PM  241123        19  ACG North America LLC
10  Jul  5 2022  2:54PM  241122        22      NBTY Global, Inc.
11  Jul  5 2022  2:50PM  241120        19    AdvaGen Pharma, Ltd
23  Jul  5 2022  2:45PM  241119        16                Seqirus
24  Jul  5 2022  2:43PM  241118        10      ISDIN Corporation
32  Jul  5 2022  2:36PM  241110        10       Methapharm, Inc.
40  Jul  5 2022  2:35PM  241116        19    AdvaGen Pharma, Ltd
63  Jul  5 2022  2:31PM  241115        10            Emerginnova

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Jul  5 2022  3:04PM  241133        10       Methapharm, Inc.             
1  Jul  5 2022  2:58PM  241125        10         Citieffe, Inc.             
2  Jul  5 2022  2:57PM  241123        19  ACG North America LLC             
3  Jul  5 2022  2:54PM  241122        22      NBTY Global, Inc.             
4  Jul  5 2022  2:50PM  241120        19    AdvaGen Pharma, Ltd             
5  Jul  5 2022  2:45PM  241119        16                Seqirus             
6  Jul  5 2022  2:43PM  241118        10      ISDIN Corporation             
7  Jul  5 2022  2:36PM  241110        10       Methapharm, Inc.             
8  Jul  5 2022  2:35PM  241116        19    AdvaGen Pharma, Ltd             
9  Jul  5 2022  2:31PM  241115        10            Emerginnova             

  Executing Released  
0                  7  
1                  2  
2                  1  
3                  1  
4                 12  
5                  1  
6                  8  
7                  8  
8                 23  
9                  4

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Jul  5 2022  3:04PM  241133        10       Methapharm, Inc.        7   
1  Jul  5 2022  2:58PM  241125        10         Citieffe, Inc.        2   
2  Jul  5 2022  2:57PM  241123        19  ACG North America LLC        1   
3  Jul  5 2022  2:54PM  241122        22      NBTY Global, Inc.        1   
4  Jul  5 2022  2:50PM  241120        19    AdvaGen Pharma, Ltd       12   
5  Jul  5 2022  2:45PM  241119        16                Seqirus        1   
6  Jul  5 2022  2:43PM  241118        10      ISDIN Corporation        8   
7  Jul  5 2022  2:36PM  241110        10       Methapharm, Inc.        8   
8  Jul  5 2022  2:35PM  241116        19    AdvaGen Pharma, Ltd       23   
9  Jul  5 2022  2:31PM  241115        10            Emerginnova        4   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Jul  5 2022  3:04PM  241133        10       Methapharm, Inc.        7   
1  Jul  5 2022  2:58PM  241125        10         Citieffe, Inc.        2   
2  Jul  5 2022  2:57PM  241123        19  ACG North America LLC        1   
3  Jul  5 2022  2:54PM  241122        22      NBTY Global, Inc.        1   
4  Jul  5 2022  2:50PM  241120        19    AdvaGen Pharma, Ltd       12   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Jul  5 2022  3:04PM  241133        10       Methapharm, Inc.       7.0   
1  Jul  5 2022  2:58PM  241125        10         Citieffe, Inc.       2.0   
2  Jul  5 2022  2:57PM  241123        19  ACG North America LLC       1.0   
3  Jul  5 2022  2:54PM  241122        22      NBTY Global, Inc.       1.0   
4  Jul  5 2022  2:50PM  241120        19    AdvaGen Pharma, Ltd      12.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Jul  5 2022  3:04PM  241133        10       Methapharm, Inc.       7.0   
1  Jul  5 2022  2:58PM  241125        10         Citieffe, Inc.       2.0   
2  Jul  5 2022  2:57PM  241123        19  ACG North America LLC       1.0   
3  Jul  5 2022  2:54PM  241122        22      NBTY Global, Inc.       1.0   
4  Jul  5 2022  2:50PM  241120        19    AdvaGen Pharma, Ltd      12.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                          
10         10124676    1297.0       39.0       64.0
12          2169787      47.0       46.0        0.0
15           241073      11.0        8.0        0.0
16           482219       2.0        0.0        0.0
19           723359      36.0        0.0        0.0
20          1205453     104.0       24.0       12.0
22           241122       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse        id  Released  Executing  Completed
0        10  10124676    1297.0       39.0       64.0
1        12   2169787      47.0       46.0        0.0
2        15    241073      11.0        8.0        0.0
3        16    482219       2.0        0.0        0.0
4        19    723359      36.0        0.0        0.0
5        20   1205453     104.0       24.0       12.0
6        22    241122       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10    1297.0       39.0       64.0
1        12      47.0       46.0        0.0
2        15      11.0        8.0        0.0
3        16       2.0        0.0        0.0
4        19      36.0        0.0        0.0
5        20     104.0       24.0       12.0
6        22       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status   Value
0        10  Released  1297.0
1        12  Released    47.0
2        15  Released    11.0
3        16  Released     2.0
4        19  Released    36.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse      10    12    15   16    19     20   22
Status                                              
Completed    64.0   0.0   0.0  0.0   0.0   12.0  0.0
Executing    39.0  46.0   8.0  0.0   0.0   24.0  0.0
Released   1297.0  47.0  11.0  2.0  36.0  104.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status      10    12    15   16    19     20   22
0          Completed    64.0   0.0   0.0  0.0   0.0   12.0  0.0
1          Executing    39.0  46.0   8.0  0.0   0.0   24.0  0.0
2           Released  1297.0  47.0  11.0  2.0  36.0  104.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status      10    12    15   16    19     20   22
0  Completed    64.0   0.0   0.0  0.0   0.0   12.0  0.0
1  Executing    39.0  46.0   8.0  0.0   0.0   24.0  0.0
2   Released  1297.0  47.0  11.0  2.0  36.0  104.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()